In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langgraph.graph.message import AnyMessage
from dotenv import load_dotenv
load_dotenv()
import os
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langgraph.graph.message import add_messages, MessagesState
from typing import Annotated
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver
from langchain_community.document_loaders import WikipediaLoader
from langchain_community.tools import DuckDuckGoSearchRun
import operator
from pydantic import BaseModel
from pydantic import Field
from typing import List
# from langgraph.types import Send
from langgraph.constants import Send

/Users/abdulvajid/AI/PROJECTS/Simple-Multi-turn-ChatBot-with-langgraph/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
)


In [3]:
msg = [SystemMessage(content='You are a helpfull Ai assistent'),
       HumanMessage(content="hai , could you help me?")]
model.invoke(msg)

AIMessage(content="Hi there! Yes, I'd love to help. What do you need assistance with? Just let me know what's on your mind.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--e1843afd-6a55-47dc-9b24-a27d4eefa211-0', usage_metadata={'input_tokens': 15, 'output_tokens': 31, 'total_tokens': 46, 'input_token_details': {'cache_read': 0}})

In [4]:
# Schema for the analyst
class Profile(BaseModel):
    name:str = Field(description='name of the analyst')
    expertise:str = Field(description='the field where the analyst specialized in')
    job:str = Field(description='job of the analyst')  

In [5]:
class Analysts(BaseModel):
    analysts: list[Profile]

In [6]:
# Overall general State
class OverallState(MessagesState):
    max_analyst: int
    analysts: list[Analysts]
    context:Annotated[list, add_messages]

In [7]:
def create_analysts(state: OverallState) -> OverallState:
    system_prompt = """You are an intelligent AI assistant specialized in generating different expertise detailed analyst profiles based on a specific topic specified by user.
Generate {max_analyst} anlyste profiles.
    """
    system_prompt = [SystemMessage(content=system_prompt.format(max_analyst=state['max_analyst']))]
    sturctured_output_model = model.with_structured_output(Analysts)
    messages = system_prompt + state['messages'] # list concat
    analysts = sturctured_output_model.invoke(messages)
    return {'analysts':analysts}

In [8]:
graph = StateGraph(OverallState)

# Nodes
graph.add_node(create_analysts,'create_analysts')

# Edges
graph.add_edge(START, 'create_analysts')
graph = graph.compile()


In [9]:
graph.invoke({'messages':[HumanMessage(content='ai in education')],'max_analyst':5})

{'messages': [HumanMessage(content='ai in education', additional_kwargs={}, response_metadata={}, id='f8cc7358-7596-4a29-9bb5-925574210646')],
 'max_analyst': 5,
 'analysts': Analysts(analysts=[Profile(name='Dr. Anya Sharma', expertise='AI in personalized learning', job='Educational Data Scientist'), Profile(name='Kenji Tanaka', expertise='AI-driven assessment', job='Assessment Technology Specialist'), Profile(name='Maria Rodriguez', expertise='Natural Language Processing for Education', job='NLP Research Scientist'), Profile(name='Dr. Ben Carter', expertise='AI ethics in education', job='AI Ethics Consultant'), Profile(name='Emily Chen', expertise='AI for special education', job='Assistive Technology Specialist')]),
 'context': []}